In [1]:
!pip -q install langchain openai tiktoken cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.4 MB/s eta 0:00:00


In [2]:
# Setup your OpenAI Api key here
import os
os.environ["OPENAI_API_KEY"] = ""

In [58]:
from langchain.chat_models import ChatOpenAI
chat_llm = ChatOpenAI(temperature=0.0)

In [59]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [60]:
merchant_schema = ResponseSchema(name="transaction_merchant",
                             description="This is the merchant")
response_schemas = [merchant_schema]

In [61]:
# The Parser would ensure we get the data in a specific format
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [62]:
format_instructions = output_parser.get_format_instructions()

In [63]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"transaction_merchant": string  // This is the merchant
}
```


In [64]:
# Prompt
template_string = """
what is the most important part of this text to help me underrstand the category of the transaction.

transaction description: ```{transaction_description}```

{format_instructions}
"""

In [69]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.0)

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template_string)
    ],
    input_variables=["transaction_description"],
    partial_variables={"format_instructions": format_instructions},
    output_parser=output_parser # here we add the output parser to the Prompt template
)

In [70]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm,
                 prompt=prompt)

In [76]:
response = chain.predict_and_parse(transaction_description="POS XXXXXXXXXXXX0001 APOLLO PHARMACY")

In [77]:
response

{'transaction_merchant': 'APOLLO PHARMACY'}